In [1]:
import zipfile as zp
from math import ceil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score, GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
import warnings
from sklearn.base import BaseEstimator, TransformerMixin
warnings.filterwarnings("ignore")

In [2]:
df_list = []
for file in ["train.csv", "test.csv"]:
    with zp.ZipFile("./data.zip") as myzip:
        with myzip.open(file) as myfile:
            df_list.append(pd.read_csv(myfile))
            
train_df, test_df = df_list

In [3]:
# Correcting variables
train_df.rename({"Parents or siblings infected": "Parents_Children_Infected", 
                 "Wife/Husband or children infected": "Partner_Siblings_Infected"}, axis=1, inplace=True)
test_df.rename({"Parents or siblings infected": "Parents_Children_Infected", 
                "Wife/Husband or children infected": "Partner_Siblings_Infected"}, axis=1, inplace=True)

In [4]:
train_df.set_index("Patient_ID", inplace=True)
test_df.set_index("Patient_ID", inplace=True)

In [5]:
X_train = train_df.drop('Deceased', axis=1)
y_train = train_df['Deceased']
X_test = test_df.copy()

In [6]:
X_train.City.fillna(value="Santa Fe", inplace=True)

In [7]:
X_train.Medical_Tent = X_train.Medical_Tent.fillna(value="NK")
test_df.Medical_Tent = test_df.Medical_Tent.fillna(value="NK")

In [8]:
X_train['Title'] = X_train['Name'].str.split('\\W', 1, expand=True)[0]
test_df['Title'] = test_df['Name'].str.split('\\W', 1, expand=True)[0]

In [9]:
X_train['Title_binary'] = X_train['Title'].apply(lambda x: 1 if x in ["Master","Miss"] else 0)
test_df['Title_binary'] = test_df['Title'].apply(lambda x: 1 if x in ["Master","Miss"] else 0)

In [10]:
def scale_function(X_data, columns_to_norm, columns_not_norm, scaler_type, scaler=None):
    '''Function to apply normalization'''
    
    if scaler_type=='MinMaxScaler':
        if scaler is None:
            scaler = MinMaxScaler().fit(X_data[columns_to_norm])
        X_scaled = scaler.transform(X_data[columns_to_norm])
        
    else:
        if scaler is None:
            scaler = StandardScaler().fit(X_data[columns_to_norm])
        X_scaled = scaler.transform(X_data[columns_to_norm])
        
    X = np.append(X_data[columns_not_norm], X_scaled, axis=1)
    X = pd.DataFrame(X, columns = columns_not_norm+columns_to_norm)

    return X, scaler

In [11]:
scaler_type="MinMaxScaler"

In [12]:
def knn_imputer_birthday(X_train, test, scaler_type):
    cols_drop = ['Family_Case_ID','Name','Medical_Expenses_Family','Medical_Tent','City']
    columns_to_norm = ['Severity','Parents_Children_Infected','Partner_Siblings_Infected']
    columns_not_norm=['Birthday_year','Title_binary']
    
    X_use = X_train.copy()
    X_test = test.copy()
    
    fill_birthday_train, scaler = scale_function(X_use.drop(columns=cols_drop), columns_to_norm=columns_to_norm,
                                                 columns_not_norm=columns_not_norm, scaler_type=scaler_type)
    fill_birthday_test, scaler = scale_function(X_test.drop(columns=cols_drop), columns_to_norm=columns_to_norm,
                                                 columns_not_norm=columns_not_norm, scaler_type=scaler_type)
    
    imputer = KNNImputer(n_neighbors=3, weights='distance').fit(fill_birthday_train)
    
    X_imputted_birthday_train = imputer.transform(fill_birthday_train)
    X_imputted_birthday_test = imputer.transform(fill_birthday_test)
    
    X_use.Birthday_year = list(pd.DataFrame(X_imputted_birthday_train).iloc[:,0]) # 0 is the index of Birthday_year
    X_test.Birthday_year = list(pd.DataFrame(X_imputted_birthday_test).iloc[:,0]) # 0 is the index of Birthday_year
    
    return X_use, X_test

In [13]:
X_train, test_df = knn_imputer_birthday(X_train, test_df, scaler_type=scaler_type)

In [14]:
X_train.Birthday_year = X_train.Birthday_year.round(0).astype(int)
test_df.Birthday_year = test_df.Birthday_year.round(0).astype(int)

In [15]:
X_train.head()

,Family_Case_ID,Severity,Name,Birthday_year,Parents_Children_Infected,Partner_Siblings_Infected,Medical_Expenses_Family,Medical_Tent,City,Title,Title_binary
Patient_ID,,,,,,,,,,,
1,4696,3,Miss Linda Betty,2005,0,0,225,NK,Santa Fe,Miss,1
2,21436,1,Ms. Ramona Elvira,1966,0,1,1663,NK,Albuquerque,Ms,0
3,7273,3,Mr. Mario Vernon,1982,0,0,221,NK,Santa Fe,Mr,0
4,8226,3,Mr. Hector Joe,1997,0,0,220,NK,Santa Fe,Mr,0
5,19689,3,Ms. Jennie Debra,1994,0,0,222,NK,Santa Fe,Ms,0


# Pipeline

In [ ]:
# Saving parameter grid for specific Grid Search run
class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (StandardScaler, MinMaxScaler, OneHotEncoder, NeighborhoodComponentsAnalysis, SelectFromModel, DecisionTreeClassifier, PCA, np.ndarray, dict)):  # Include all classes that aren't serializable here
            return str(obj)
        # Let the base class default method raise the TypeError 
        return json.JSONEncoder.default(self, obj)
    

def gs_outputs(id_num, gscv_fitted, grid):
    """
    Function that saves information of each grid-search.
    
    id_num: takes an id_number which identifies the grid-search
    gscv_fitted: takes a fitted GridSearchCV object
    grid: takes the grid used to fit the GridSearchCV object
    
    Returns:
    top 20 configurations hyper-parameter presence graphic
    (also outputs the "logs" of each grid-search to the output directory)
    """
    # Saving parameter grid for specific Grid Search run
    with open("./outputs/grids.txt", "a") as file:
        file.write("# {}------------------------------------------------------ #\n".format(id_num) + json.dumps(grid, cls=ComplexEncoder) + "\n\n")

    # Saving cv_results for specific Grid Search run
    score_summary = pd.DataFrame(gscv_fitted.cv_results_).sort_values(by="mean_test_score", ascending=False)
    score_summary.to_csv("./outputs/grid_search_results{}.csv".format(id_num))
    
    # Assessing distribution of hyper-parameter values amongst top 20 models
    sns.set()

    # Features to plot
    plot_features = list(map(lambda x: "param_" + x, {i for j in range(len(grid)) for i in grid[j].keys()}))
    plot_df = score_summary.reset_index(drop=True).loc[:19, :].fillna("NaN")

    # figure and axes
    fig, axes = plt.subplots(3, ceil(len(plot_features)/3), figsize=(23,13))

    # plot data
    for ax, x in zip(axes.flatten(), plot_features):
        try:
            sns.countplot(x=x, data=plot_df, ax=ax)
        except TypeError:
            sns.countplot(x=plot_df[x].apply(json.dumps), ax=ax)

    plt.suptitle("Hyper-parameter presence on top 20 models", y=0.95, fontsize=25)

    plt.savefig("./outputs/grid_results{}.png".format(id_num))
    
    # Model training - using best parameters to train model on entire data for submission
    best_model = full_pipeline.set_params(**gscv_fitted.best_params_)
    best_model.fit(X, y)  # Using all of the data to fit the model

    # Predicting the 
    y_pred = best_model.predict(test_df)

    # Submission
    pd.DataFrame(data={"Employee_ID": test_df.index.to_list(), "Churn_risk": y_pred}).to_csv("./outputs/submission{}.csv".format(id_num), index=False)
    
    return plt.show()


def feature_importance(id_num, gscv_fitted, fimp_pre=True):
    """
    Function that show feature importance plot and saves it to outputs folder
    
    id_num: takes an id_number which identifies the grid-search
    gscv_fitted: takes a fitted GridSearchCV object
    fimp_pre: whether to evaluate feature importance pre or pos model
    
    Returns:
    feature importance graphic    
    """
    if fimp_pre:
        if (gscv_fitted.named_steps["prep"].named_steps["fselect1"]) and (gscv_fitted.named_steps["prep"].named_steps["fselect1"] != 'passthrough'):

            # Get feature names
            numeric_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[0][2]
            categorical_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[1][2]
            categorical_features = list(gscv_fitted.named_steps["prep"].named_steps["join_features"].named_transformers_["categorical_pipeline"].named_steps["one_hot_encoder"].get_feature_names(categorical_features))
            feature_names = metric_features + categorical_features

            # Get feature importances
            if hasattr(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_, "feature_importances_"):  # Check if features_importances_ attribute exists. Necessary when there's tree selectors
                if len(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.feature_importances_.shape) != 2: 
                    feature_importances = pd.DataFrame(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.feature_importances_, index=feature_names).T
                else:
                    feature_importances = pd.DataFrame(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.feature_importances_, index=gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.classes_, columns=feature_names)
            else:
                if len(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.coef_.shape) != 2: 
                    feature_importances = pd.DataFrame(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.coef_, index=feature_names).T
                else:
                    feature_importances = pd.DataFrame(gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.coef_, index=gscv_fitted.named_steps["prep"].named_steps["fselect1"].estimator_.classes_, columns=feature_names)

            colors = feature_importances.applymap(lambda x: "tab:red" if x < 0 else "tab:blue")  # Get positive and negative colors
            threshold = gscv_fitted.named_steps["prep"].named_steps["fselect1"].threshold_  # Get SelectFromModel threshold value 

            sns.set()
            if feature_importances.shape[0] == 3:
                fig, axes = plt.subplots(3, 1, figsize=(23,12))
                blue_patch = mpatches.Patch(color='tab:blue', label='Positive')
                red_patch = mpatches.Patch(color='tab:red', label='Negative')
                threshold_line = mlines.Line2D([], [], linestyle="dashed", color='black', label='Threshold')

                for i, ax in enumerate(axes.flatten()):
                    ax.bar(x=feature_names, height=abs(feature_importances.iloc[i]), color=colors.iloc[i])
                    ax.hlines(threshold, -0.5, 18.5, linestyles="dashed")
                    ax.set_xticklabels(feature_names, rotation = 30, ha="right")
                    ax.set_title("Class {} Feature Importance".format(feature_importances.index[i]), fontsize=18)
                    ax.legend(handles=[blue_patch, red_patch, threshold_line], fontsize=12)

                plt.subplots_adjust(hspace=0.6)
                plt.savefig("./outputs/feature_importance{}.png".format(id_num))
                return plt.show()
            else: 
                fig = plt.figure(figsize=(23,6))
                blue_patch = mpatches.Patch(color='tab:blue', label='Positive')
                red_patch = mpatches.Patch(color='tab:red', label='Negative')
                threshold_line = mlines.Line2D([], [], linestyle="dashed", color='black', label='Threshold')

                plt.bar(x=feature_names, height=abs(feature_importances.iloc[0]), color=colors.iloc[0])
                plt.hlines(threshold, -0.5, len(feature_names) - 0.5, linestyles="dashed")
                plt.xticks(rotation=30, ha="right")
                plt.title("Pre Feature Importance", fontsize=18)
                plt.legend(handles=[blue_patch, red_patch, threshold_line], fontsize=12)

                plt.savefig("./outputs/feature_importance_pre{}.png".format(id_num))
                return plt.show()
        else:
            print("No feature selection was done!")
    else:
        if (gscv_fitted.named_steps["prep"].named_steps["fselect2"]) and (gscv_fitted.named_steps["prep"].named_steps["fselect2"] != 'passthrough'):
            # Obtaining Component Names
            feature_names = ["PC" + str(i) for i in range(1, best_model.named_steps["prep"].named_steps["fselect2"].n_components + 1)]
        else:  
            # Get feature names
            numeric_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[0][2]
            categorical_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[1][2]
            categorical_features = list(gscv_fitted.named_steps["prep"].named_steps["join_features"].named_transformers_["categorical_pipeline"].named_steps["one_hot_encoder"].get_feature_names(categorical_features))
            feature_names = metric_features + categorical_features

            # Get filtered feature names
            if (gscv_fitted.named_steps["prep"].named_steps["fselect1"]) and (gscv_fitted.named_steps["prep"].named_steps["fselect1"] != 'passthrough'):
                feature_names = np.array(feature_names)[gscv_fitted.named_steps["prep"].named_steps["fselect1"].get_support()]

        # Get feature importances
        if len(gscv_fitted.named_steps["model"].feature_importances_.shape) != 2: 
            feature_importances = pd.DataFrame(gscv_fitted.named_steps["model"].feature_importances_, index=feature_names).T
        else:
            feature_importances = pd.DataFrame(gscv_fitted.named_steps["model"].feature_importances_, index=gscv_fitted.named_steps["model"].classes_, columns=feature_names)
        
        colors = feature_importances.applymap(lambda x: "tab:red" if x < 0 else "tab:blue")  # Get positive and negative colors

        sns.set()
        fig = plt.figure(figsize=(23,6))
        blue_patch = mpatches.Patch(color='tab:blue', label='Positive')
        red_patch = mpatches.Patch(color='tab:red', label='Negative')

        plt.bar(x=feature_names, height=abs(feature_importances.iloc[0]), color=colors.iloc[0])
        plt.xticks(rotation=30, ha="right")
        plt.title("Pos Feature Importance", fontsize=18)
        plt.legend(handles=[blue_patch, red_patch], fontsize=12)

        plt.savefig("./outputs/feature_importance_pos{}.png".format(id_num))
        return plt.show()
        

def plot_tree(id_num, gscv_fitted):
    """
    Function that shows the decision tree graph and saves it as png
    
    id_num: takes an id_number which identifies the grid-search
    gscv_fitted: takes a fitted GridSearchCV object
    
    Returns:
    decision tree graphic
    """
    if (gscv_fitted.named_steps["prep"].named_steps["fselect2"]) and (gscv_fitted.named_steps["prep"].named_steps["fselect2"] != 'passthrough'):
        # Obtaining Component Names
        feature_names = ["PC" + str(i) for i in range(1, best_model.named_steps["prep"].named_steps["fselect2"].n_components + 1)]
    else:  
        # Get feature names
        numeric_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[0][2]
        categorical_features = gscv_fitted.named_steps["prep"].named_steps["join_features"].transformers_[1][2]
        categorical_features = list(gscv_fitted.named_steps["prep"].named_steps["join_features"].named_transformers_["categorical_pipeline"].named_steps["one_hot_encoder"].get_feature_names(categorical_features))
        feature_names = metric_features + categorical_features

        # Get filtered feature names
        if (gscv_fitted.named_steps["prep"].named_steps["fselect1"]) and (gscv_fitted.named_steps["prep"].named_steps["fselect1"] != 'passthrough'):
            feature_names = np.array(feature_names)[gscv_fitted.named_steps["prep"].named_steps["fselect1"].get_support()]
    
    # Set PATH variable to locate graphviz executables
    if os.environ['PATH'].split(";")[-1] != 'C:\\Users\\davids\\Anaconda3\\envs\\ml\\Library\\bin\\graphviz':
        os.environ['PATH'] = os.environ['PATH'] + ';' + os.environ['CONDA_PREFIX'] + r"\Library\bin\graphviz"
    
    # Export a decision tree in DOT format
    dot_data = export_graphviz(gscv_fitted.named_steps["model"],
                               feature_names=feature_names,  
                               class_names=gscv_fitted.named_steps["model"].classes_,
                               filled=True,
                               rounded=True)
    
    pydot_graph = pydotplus.graph_from_dot_data(dot_data)  # load graph in dot format
    pydot_graph.set_size('"50,50"')  # set size of graph figure
    pydot_graph.write_png("./outputs/decision_tree{}.png".format(id_num))  # output graph as png
    return graphviz.Source(pydot_graph.to_string())


def effective_alpha_analysis(id_num, gscv_fitted, X_train, y_train, max_alpha, min_alpha=0):
    """
    Function that analyses the alphas of a decision tree and plots the cross-validated mean alpha performance of default trees.
    
    id_num: takes an id_number which identifies the grid-search
    gscv_fitted: takes a fitted GridSearchCV object
    X_train: raw train data
    y_train: train data labels
    max_alpha: max alpha consider in cross-validation
    min_alpha: min alpha consider in cross-validation
    
    Returns:
    alpha analysis graphic
    """
    original_model = clone(gscv_fitted.named_steps["model"])
    analyze_model = clone(gscv_fitted.named_steps["model"].set_params(min_weight_fraction_leaf=0.0, max_depth=None))

    # Transforming X_train based on fitted preparation pipeline
    X_train_trans = gscv_fitted.named_steps["prep"].transform(X_train)

    # Producing cost_complexity_pruning_path of default tree on X_train_trans
    path = analyze_model.cost_complexity_pruning_path(X_train_trans, y_train)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities

    # Defininf a default tree for each ccp_alpha value found
    ccp_alphas_delim = ccp_alphas[(ccp_alphas >= min_alpha) & (ccp_alphas <= max_alpha)]
    trees = [clone(analyze_model.set_params(ccp_alpha=ccp_alpha)) for ccp_alpha in ccp_alphas_delim]

    # Producing 5-fold cv f1-scores for each tree on X_train_trans
    cv_scores = np.array([cross_val_score(tree, X_train_trans, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=1), scoring='f1_micro', n_jobs=-1) for tree in trees])
    cv_scores_mean, cv_scores_std = cv_scores.mean(axis=1), cv_scores.std(axis=1)
    cv_scores_ci95 = 1.96 * cv_scores_std / cv_scores_mean
    global_optimum = [ccp_alphas_delim[np.argmax(cv_scores_mean)], np.max(cv_scores_mean)]
    original_cv_score = cross_val_score(original_model, X_train_trans, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=1), scoring='f1_micro', n_jobs=-1).mean()
    
    sns.set()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23, 8))

    # Plot Total Impurity vs effective alpha for training set
    ax1.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
    ax1.set_xlabel("effective alpha", fontsize=14)
    ax1.set_ylabel("total impurity of leaves", fontsize=14)
    ax1.set_title("Total Impurity vs effective alpha for training set", fontsize=18)

    # Plot F1-score vs alpha for cross-validation score
    ax2.plot(ccp_alphas_delim, cv_scores_mean, marker='o', label="AVG Scores", drawstyle="steps-post")
    ax2.plot(global_optimum[0], global_optimum[1], 'o', label="G.O.: [{0:.6f}, {1:.3f}]".format(global_optimum[0], global_optimum[1]))
    ax2.fill_between(ccp_alphas_delim, (cv_scores_mean - cv_scores_ci95), (cv_scores_mean + cv_scores_ci95), color='b', alpha=.1)
    ax2.hlines(original_cv_score, min_alpha, max_alpha, linestyles="dashed", label="Original Model")
    ax2.set_xlim(left=min_alpha, right=max_alpha)
    ax2.set_xlabel("alpha", fontsize=14)
    ax2.set_ylabel("f1-score", fontsize=14)
    ax2.set_title("F1-score vs alpha for cross-validation score", fontsize=18)
    ax2.legend()
    
    plt.savefig("./outputs/alpha_analysis{}.png".format(id_num))
    return plt.show()

In [24]:
# This cell holds all the Custom Transformers we designed to pipeline the raw data to the model
class PrepImpute(BaseEstimator, TransformerMixin):
    """
    Creates Title
    Creates Title_binary
    Fills Medical_Tent with "NK"
    Imputes City with most frequent value
    """
    #Class constructor method that takes in a list of values as its argument
    def __init__(self):
        self.city_imputer = SimpleImputer(strategy="most_frequent")
        
    #Return self nothing else to do here
    def fit(self, X, y=None):
        self.city_imputer.fit(X[["City"]])
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X):
        
        X2 = X.copy()
        
        # CREATE NEW VARIABLES
        X2['Title'] = X2['Name'].str.split('\\W', 1, expand=True)[0]
        X2['Title_binary'] = X2['Title'].apply(lambda x: 1 if x in ["Master","Miss"] else 0)
        X2["Medical_Tent"] = X2["Medical_Tent"].fillna(value="NK")
        X2['City'] = self.city_imputer.transform(X2[["City"]])
        
        return X2 
    

class KNNImputerScaled(KNNImputer):
    """
    KNNImputer subclass that scales variables before applying imputation
    Impute Birthday_year using scaled and non-scaled variables with KNNImputer
    """
    def __init__(self, missing_values=np.nan, n_neighbors=5, weights='uniform', metric='nan_euclidean', copy=True, add_indicator=False,
                 columns_to_norm=['Severity','Parents_Children_Infected','Partner_Siblings_Infected'], columns_not_to_norm=['Birthday_year','Title_binary'],
                 scaler=MinMaxScaler(), **scaler_args):
        super().__init__(missing_values=missing_values, 
                         n_neighbors=n_neighbors, 
                         weights=weights,
                         metric=metric,
                         copy=copy, 
                         add_indicator=add_indicator)
        self.scaler = scaler.set_params(**scaler_args)
        self.columns_to_norm = columns_to_norm
        self.columns_not_to_norm = columns_not_to_norm
        
    def fit(self, X, y=None):
        X_scaled = self.scaler.fit_transform(X[self.columns_to_norm])
        X_trans = np.concatenate([X_scaled, X[self.columns_not_to_norm]], axis=1)
        super().fit(X_trans)
        return self
    
    def transform(self, X, y=None):
        X_scaled = self.scaler.transform(X[self.columns_to_norm])
        X_trans = np.concatenate([X_scaled, X[self.columns_not_to_norm]], axis=1)
        X_imputed_scaled = pd.DataFrame(super().transform(X_trans), index=X.index, columns=self.columns_to_norm + self.columns_not_to_norm)
        X_imputed = X.copy()
        X_imputed["Birthday_year"] = X_imputed_scaled["Birthday_year"].round(0).astype(int)
        
        return X_imputed
    
    
class FeatureEngineering(BaseEstimator, TransformerMixin):
    """
    Creates Age
    Creates Gender
    Ceates Parents_Children_Infected_Binary
    Creates Partner_Siblings_Infected_Binary
    Creates Medical_Tent_Binary
    Creates Family_Infected
    Creates Family_Infected_Binary
    Creates Family_Deceased
    Creates Family_Deceased_Reduced
    Creates Medical_Expenses_Individual
    """
    #Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass    
        
    #Return self nothing else to do here
    def fit(self, X, y):
        self.familycount_ = X.Family_Case_ID.value_counts().to_dict()
        X_train_w_dec = pd.concat([X, y], axis=1)
        self.deceasedfamcount_ = X_train_w_dec.loc[X_train_w_dec['Deceased']==1].Family_Case_ID.value_counts().to_dict()
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X):
        X2 = X.copy()
        # CREATE NEW VARIABLES
        X2['Age'] = X2['Birthday_year'].map(lambda x: 2020 - x)
        X2['Gender'] = X2['Title'].map(lambda x: "Male" if x in ["Mr", "Master"] else "Female")
        X2['Parents_Children_Infected_Binary'] = X2['Parents_Children_Infected'].map(lambda x: 0 if x==0 else 1)  
        X2['Partner_Siblings_Infected_Binary'] = X2['Partner_Siblings_Infected'].map(lambda x: 0 if x==0 else 1)
        X2["Medical_Tent_Binary"] = X2["Medical_Tent"].map(lambda x: x if x == "NK" else "K")
        #X2["Pediatric_Binary"] = X2["Age"].map(lambda x: 1 if x < 18 else 0)
        #X2["3rd_Age_Binary"] = X2["Age"].map(lambda x: 1 if x >= 65 else 0)
        X2["Family_Infected"] = X2["Family_Case_ID"].map(self.familycount_).fillna(1)
        X2["Family_Infected_Binary"] = X2["Family_Infected"].map(lambda x: 0 if x<=1 else 1)
        X2["Family_Deceased"] = (X2["Family_Case_ID"].map(self.deceasedfamcount_) - 1).fillna(0)
        X2["Family_Deceased_Reduced"] = X2['Family_Deceased'].map(lambda x: x if x==0 else (2 if x>=3 else 1))
        #X2["Dead_infected_ratio_family"] = X2["Family_Deceased"] / X2["Family_Infected"]
        X2["Medical_Expenses_Individual"] = X2["Medical_Expenses_Family"] / X2["Family_Infected"]
        # FURTHER TRANSFORMATIONS
        X2["Medical_Expenses_Individual"][X2["Medical_Expenses_Individual"] > 3000] = 3000
        
        return X2 
    
    
class Transformations(BaseEstimator, TransformerMixin):
    """
    Transforms Medical_Expenses_Individual by applying Box-Cox power transformation
    Drops unnecessary variables
    """
    #Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass
        
    #Return self nothing else to do here
    def fit(self, X, y=None):
        self.lambda_ = boxcox(X["Medical_Expenses_Individual"] + 1)[1]
        return self

    #Transformer method we wrote for this transformer 
    def transform(self, X):
        X["Medical_Expenses_Individual"] = boxcox(X["Medical_Expenses_Individual"] + 1, alpha=self.lambda_)[0]
        # DROP INTERMEDIATE VARIABLES
        X = X.drop(["Family_Case_ID", "Name", "Birthday_year", "Title", "Title_binary", "Partner_Siblings_Infected", 
                    "Medical_Tent", "Parents_Children_Infected", "Partner_Siblings_Infected_Binary", 
                    "Parents_Children_Infected_Binary", "Family_Deceased", "Family_Infected", "Medical_Expenses_Family"], axis=1)
        
        return X 

In [17]:
X_train_pipe = train_df.drop('Deceased', axis=1)
y_train_pipe = train_df['Deceased']

In [18]:
# pi = PrepImpute()
# X_trans = pi.fit_transform(X_train_pipe)
# X_trans

In [19]:
# knnimp = KNNImputerScaled(n_neighbors=3, weights='distance', scaler=MinMaxScaler)
# knnimp.fit(X_trans)
# X_scaled = knnimp.transform(X_trans)
# X_scaled

In [20]:
# # Checking that KNNImputerScaled gets the same output as the KNNImputer used outside pipeline
# (X_train.loc[train_df["Birthday_year"].isna(), "Birthday_year"] == X_scaled.loc[train_df["Birthday_year"].isna(), "Birthday_year"]).all()

In [21]:
# boxcox1, lambd = boxcox(x=X_train["Medical_Expenses_Family"]+1)
# boxcox2 = boxcox(x=(X_train["Medical_Expenses_Family"]+1), alpha=lambd)[0]
# boxcox1, boxcox2

In [22]:
# boxcox3 = boxcox(x=test_df["Medical_Expenses_Family"]+1, alpha=lambd)[0]
# boxcox3

In [51]:
# Data Cleaning Pipeline
clean_pipeline = Pipeline([
    ('prepimpute', PrepImpute()),
    ('knnimputer', KNNImputerScaled()),  # tune: n_neighbors, weights, scaler
    ('engineering', FeatureEngineering()),
    ('transformation', Transformations())
])

# Spliting features
scale_columns = ["Severity", "Age", "Family_Deceased_Reduced", "Medical_Expenses_Individual"]
ohe_columns = ["City", "Gender", "Medical_Tent_Binary", "Family_Infected_Binary"]

# Combining features and applying different transformations
join_pipeline = ColumnTransformer([('scaler', "passthrough", scale_columns),
                                   ('ohe', OneHotEncoder(sparse=False), ohe_columns)])  # tune: drop

# Feature Selection
fselect1 = SelectFromModel(LogisticRegression(penalty="l1", max_iter=400, multi_class="multinomial", solver="saga", n_jobs=-1, random_state=1))  # tune: estimator__C
fselect2 = SelectFromModel(LogisticRegression(penalty="l2", max_iter=400,  multi_class="multinomial", solver="saga", n_jobs=-1, random_state=1))  # tune: estimator__C, threshold
dimred1 = NeighborhoodComponentsAnalysis(max_iter=25, tol=0.005, random_state=1)  # tune: n_components, tol=0.005 (it takes to long)

# Full Preprocessing Pipeline
prep_pipeline = Pipeline([
    ("clean", clean_pipeline),
    ("join", join_pipeline),
    ("fselect", "passthrough"),
    ("dimred", "passthrough")
])

# Model
lr = LogisticRegression(max_iter=400, random_state=1, n_jobs=-1)
knn = KNeighborsClassifier(n_jobs=-1)

# Full Model Pipeline
full_pipeline = Pipeline([
    ("prep", prep_pipeline),
    ("model", "passthrough")
])

grid = [
    {"prep__clean__knnimputer__n_neighbors": [3, 5, 9, 17],
     "prep__clean__knnimputer__weights": ["uniform", "distance"],
     "prep__clean__knnimputer__scaler": [StandardScaler(), MinMaxScaler()],
     "prep__join__scaler": [StandardScaler(), MinMaxScaler()],
     "prep__join__ohe__drop": [None, "first"],  # define categories to drop
     "prep__fselect": [None],
     "prep__dimred": [None],
     "model": [knn],
     "model__n_neighbors": [5, 9, 17],
     "model__weights": ["uniform", "distance"]
    }
]

In [52]:
# X_trans = pd.DataFrame(prep_pipeline.fit_transform(X_train_pipe, y_train_pipe), index=X_train_pipe.index, columns=scale_columns + list(prep_pipeline.named_steps["join"].named_transformers_["ohe"].get_feature_names(ohe_columns)))
# X_trans

In [53]:
# Instatiating GridSearch
gscv = GridSearchCV(full_pipeline, grid, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=1), scoring='accuracy', verbose=1, n_jobs=-1)

# Get ID of grid search
id_num = input("Insert GridSearch ID number: ")

# Grid Search and model training
gscv.fit(X_train_pipe, y_train_pipe)

# Obtain outputs from Grid Search
# gs_outputs(id_num, gscv, grid)

Insert GridSearch ID number:  1


Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  5.2min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('prep',
                                        Pipeline(memory=None,
                                                 steps=[('clean',
                                                         Pipeline(memory=None,
                                                                  steps=[('prepimpute',
                                                                          PrepImpute()),
                                                                         ('knnimputer',
                                                                          KNNImputerScaled(add_indicator=False,
                                                                                           columns_not_to_norm=['Birthday_year',
                                                                                         

In [54]:
gscv.best_score_

0.8122222222222222

In [56]:
gscv.best_params_

{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=17, p=2,
                      weights='distance'),
 'model__n_neighbors': 17,
 'model__weights': 'distance',
 'prep__clean__knnimputer__n_neighbors': 17,
 'prep__clean__knnimputer__scaler': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'prep__clean__knnimputer__weights': 'distance',
 'prep__dimred': None,
 'prep__fselect': None,
 'prep__join__ohe__drop': None,
 'prep__join__scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}

In [58]:
X_test

,Family_Case_ID,Severity,Name,Birthday_year,Parents_Children_Infected,Partner_Siblings_Infected,Medical_Expenses_Family,Medical_Tent,City
Patient_ID,,,,,,,,,
901,49242,3,Mr. Jody Pedro,NaN,0,0,203,NaN,Santa Fe
902,10400,3,Mr. Kevin Brent,1988.0,0,0,631,NaN,Santa Fe
903,10795,3,Mr. Frankie Cary,1981.0,1,0,376,NaN,Albuquerque
904,62440,3,Mr. Rick Pete,NaN,0,1,405,NaN,Albuquerque
905,81311,2,Mr. Matthew Erick,1996.0,0,0,378,NaN,Santa Fe
...,...,...,...,...,...,...,...,...,...
1296,110522,3,Mr. Luther Rogelio,NaN,0,0,221,NaN,Santa Fe
1297,118768,3,Mr. Emanuel Ruben,NaN,0,0,202,NaN,Albuquerque
1298,86158,1,Mrs. Misty Camille,1994.0,0,1,3830,C,Albuquerque


In [64]:
pd.DataFrame(gscv.predict(X_test), index=X_test.index, columns=["Deceased"]).to_csv("./outputs/submission1.csv")